# generate DSEC-Soft GT-labelme

```
生成 labelme 格式 label

python demo/image_demo.py demo/demo.jpg \
    detection/config/yolov8_s_mask-refine_syncbn_fast_8xb16-500e_coco.py \
    detection/checkpoint/yolov8_s_mask-refine_syncbn_fast_8xb16-500e_coco_20230216_095938-ce3c1b3f.pth \
    --out-dir DESC-Soft/test \
    --class-name car \
    --to-labelme
```

## import

In [1]:
%cd ~/mmyolo/

from pathlib import Path
import json

import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

import mmcv
from mmcv.transforms import Compose
from mmdet.apis import inference_detector, init_detector
from mmengine.utils import track_iter_progress

from mmyolo.registry import VISUALIZERS


/home/jiangtianbin/mmyolo


## path config

In [2]:
ann_base_path = Path('/home/jiangtianbin/event-rgb-fusion/DSEC_detection_labels')
ann_train_path = ann_base_path / Path('train_ann.json')
ann_test_path = ann_base_path / Path('test_ann.json')
ann_val_path = ann_base_path / Path('val_ann.json')

## show all sequences

In [3]:
def read_josn(path):
    with open(path, 'r') as f:
        data = json.load(f)
        return data

def get_sequences_name(path):
    ann_data = read_josn(path)

    seq_name_set = set()

    for x in ann_data['images']:
        file_name_list = x['file_name'].split('/')
        seq_name = file_name_list[1]
        seq_name_set.add(seq_name)

    return seq_name_set

print(get_sequences_name(ann_train_path))
print(get_sequences_name(ann_test_path))

{'zurich_city_09_a', 'zurich_city_02_a', 'thun_00_a', 'zurich_city_09_c', 'zurich_city_01_c', 'zurich_city_00_b', 'zurich_city_05_a', 'zurich_city_11_a', 'zurich_city_03_a', 'zurich_city_10_a', 'zurich_city_11_b', 'interlaken_00_f', 'zurich_city_08_a', 'zurich_city_06_a', 'zurich_city_09_e', 'zurich_city_01_b', 'zurich_city_04_f', 'interlaken_00_e', 'zurich_city_01_d', 'zurich_city_09_d', 'zurich_city_02_d', 'zurich_city_04_b', 'interlaken_00_d', 'zurich_city_10_b', 'zurich_city_02_b', 'zurich_city_11_c', 'zurich_city_04_c', 'zurich_city_02_e', 'zurich_city_07_a', 'zurich_city_02_c', 'interlaken_00_c', 'zurich_city_00_a', 'zurich_city_04_d'}
{'zurich_city_01_e', 'zurich_city_04_e', 'zurich_city_04_a', 'zurich_city_09_b', 'zurich_city_01_f'}


## Set One Info [ train | test ]

In [4]:
# gt_mode = 'train'
gt_mode = 'test'

if gt_mode == 'train':
    ann_path = Path(ann_train_path)
else:
    ann_path = Path(ann_test_path)

# save_ann_file = None

## load image pathes

In [5]:
ann_data = read_josn(ann_path)

image_path_list = []

for x in ann_data['images']:
    event_path = x['file_name']
    
    seq_name = event_path.split('/')[1]
    image_path = event_path.replace('events/', 'images/').replace('event_images/', 'left/rectified/').replace('.npy', '.png')

    # base = 'data/DSEC/train/'
    # image = plt.imread(base + image_path)
    # plt.imshow(image)
    
    image_path_list.append(image_path)

# for debug:
# image_path_list = image_path_list[:2]

print(len(image_path_list))

4769


## generate label(labelme)

In [12]:
# changed from demo/image_demo.py

import os

import mmcv
from mmdet.apis import inference_detector, init_detector
from mmengine.config import Config, ConfigDict
from mmengine.logging import print_log
from mmengine.utils import ProgressBar, path

from mmyolo.registry import VISUALIZERS
from mmyolo.utils import switch_to_deploy
from mmyolo.utils.labelme_utils import LabelmeFormat
from mmyolo.utils.misc import get_file_list, show_data_classes

class InferenceArgs:
    config = 'detection/config/yolov8_s_mask-refine_syncbn_fast_8xb16-500e_coco.py'
    checkpoint = 'detection/checkpoint/yolov8_s_mask-refine_syncbn_fast_8xb16-500e_coco_20230216_095938-ce3c1b3f.pth'
    out_dir = 'DSEC-Soft/soft_labelme/' + gt_mode + '/'
    img_base_dir = 'data/DSEC/train'   # NOTE: all sequences are from DSEC training data
    device = 'cuda:0'
    show = False   # Show the detection results
    deploy = True   # Switch model to deployment mode
    tta = False   # Whether to use test time augmentation
    score_thr = 0.3   # Bbox score threshold
    class_name = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck']  # in inference, retain all posible classes
    to_labelme = True

def get_model():
    
    if InferenceArgs.to_labelme and InferenceArgs.show:
        raise RuntimeError('`--to-labelme` or `--show` only '
                           'can choose one at the same time.')
    config = InferenceArgs.config

    if isinstance(config, (str, Path)):
        config = Config.fromfile(config)
    elif not isinstance(config, Config):
        raise TypeError('config must be a filename or Config object, '
                        f'but got {type(config)}')
    if 'init_cfg' in config.model.backbone:
        config.model.backbone.init_cfg = None

    if InferenceArgs.tta:
        assert 'tta_model' in config, 'Cannot find ``tta_model`` in config.' \
            " Can't use tta !"
        assert 'tta_pipeline' in config, 'Cannot find ``tta_pipeline`` ' \
            "in config. Can't use tta !"
        config.model = ConfigDict(**config.tta_model, module=config.model)
        test_data_cfg = config.test_dataloader.dataset
        while 'dataset' in test_data_cfg:
            test_data_cfg = test_data_cfg['dataset']

        # batch_shapes_cfg will force control the size of the output image,
        # it is not compatible with tta.
        if 'batch_shapes_cfg' in test_data_cfg:
            test_data_cfg.batch_shapes_cfg = None
        test_data_cfg.pipeline = config.tta_pipeline

    # TODO: TTA mode will error if cfg_options is not set.
    #  This is an mmdet issue and needs to be fixed later.
    # build the model from a config file and a checkpoint file
    model = init_detector(
        config, InferenceArgs.checkpoint, device=InferenceArgs.device, cfg_options={})

    if InferenceArgs.deploy:
        switch_to_deploy(model)

    if not InferenceArgs.show:
        path.mkdir_or_exist(InferenceArgs.out_dir)

    # get model class name
    dataset_classes = model.dataset_meta.get('classes')

    # ready for labelme format if it is needed
    to_label_format = LabelmeFormat(classes=dataset_classes)

    # check class name
    if InferenceArgs.class_name is not None:
        for class_name in InferenceArgs.class_name:
            if class_name in dataset_classes:
                continue
            show_data_classes(dataset_classes)
            raise RuntimeError(
                'Expected InferenceArgs.class_name to be one of the list, '
                f'but got "{class_name}"')
        
    return model, to_label_format

def generate_gt(img_path):

    result = inference_detector(model, img_path)

    seq_name = img_path.split('/')[4]
    # print(seq_name)
    filename = seq_name + '-' + os.path.basename(img_path)
    # print(filename)
    out_file = os.path.join(InferenceArgs.out_dir, filename)

    # Get candidate predict info with score threshold
    pred_instances = result.pred_instances[
        result.pred_instances.scores > InferenceArgs.score_thr]

    if InferenceArgs.to_labelme:
        # save result to labelme files
        out_file = out_file.replace(
            os.path.splitext(out_file)[-1], '.json')
        to_label_format(pred_instances, result.metainfo, out_file,
                        InferenceArgs.class_name)
        


model, to_label_format = get_model()

progress_bar = ProgressBar(len(image_path_list))
for path in image_path_list:
    img_path = os.path.join(InferenceArgs.img_base_dir, path)
    generate_gt(img_path)
    progress_bar.update()

Loads checkpoint by local backend from path: detection/checkpoint/yolov8_s_mask-refine_syncbn_fast_8xb16-500e_coco_20230216_095938-ce3c1b3f.pth
Switch model to deploy modality.
[                                                  ] 0/2, elapsed: 0s, ETA:01/02 11:22:36 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
01/02 11:22:36 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2/2, 0.4 task/s, elapsed: 5s, ETA:     0s

/home/jiangtianbin/miniconda3/envs/mmyolo/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343962757/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
